# Build Docker Image

In this notebook, we will build the docker container that contains the  model, Flask web application, model driver and all dependencies. Make sure you have logged in using docker login.

In [ ]:
dockerlogin='YOUR_DOCKER_LOGIN'
imagerepo='/mlaksdep'

In [ ]:
import os
from os import path
import json
import shutil
import dotenv
%load_ext dotenv

We will be using the following Docker information to push the image to docker hub. Please modify the values in the `set_key` statements as you see fit.

In [ ]:
dotenv_path = os.path.join('.', '.env')  # The location of the dotenv file
if os.path.isfile(dotenv_path):          # Remove any pre-existing dotenv file to ensure a blank slate
    os.remove(dotenv_path)
with open(dotenv_path, 'w'):             # Create an empty dotenv file
    None
# Your docker login and image repository name
dotenv.set_key(dotenv_path, 'docker_login', dockerlogin)
dotenv.set_key(dotenv_path, 'image_repo', imagerepo)

Import the values into the browser's environment.

In [ ]:
%dotenv -o

In [ ]:
os.makedirs('flaskwebapp', exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'nginx'), exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'etc'), exist_ok=True)

In [ ]:
shutil.copy('driver.py', 'flaskwebapp')
shutil.copy('duplicate_model.py', 'flaskwebapp')
shutil.copy('model.pkl', 'flaskwebapp')
shutil.copy('questions.tsv', 'flaskwebapp')
shutil.copy('ItemSelector.py', 'flaskwebapp')
os.listdir('flaskwebapp')

In [ ]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    print(response)
    return json.dumps({'result': response})


@app.route("/")
def healthy():
    return "Healthy"

# LightGBM Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

In [ ]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Here, we write the configuration for the Nginx which creates a proxy between ports 80 and 5000.

In [ ]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

In [ ]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

In [ ]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

In [ ]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

We now create a custom image and install all the necessary dependencies.

In [ ]:
%%writefile flaskwebapp/requirements.txt

pandas==0.23.3
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.18.4

In [ ]:
%%writefile flaskwebapp/dockerfile

FROM ubuntu:16.04

USER root
RUN mkdir /code
WORKDIR /code
RUN chmod -R a+w /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        wget \
        curl \
        git \
        supervisor \
        nginx && \
    rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ 

ENV PYTHON_VERSION=3.5
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy pyyaml scipy \
    ipython jupyter ipykernel && \
    /opt/conda/bin/conda clean -ya
    
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
    
RUN pip install --upgrade pip && \
    pip install scikit-learn==0.19.1 && \
    pip install lightgbm==2.1.2 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 8888
EXPOSE 5000
EXPOSE 80

CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

The image name below refers to our dockerhub account. If you wish to push the image to your account make sure you change the docker login.

In [ ]:
image_name = os.getenv('docker_login') + os.getenv('image_repo')
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

This step build the docker image prior to pushing it. It can take as long as 10 minutes on a Standard NC6 DLVM.

In [ ]:
%%time
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

In [ ]:
print('Docker image name {}'.format(image_name))

In [ ]:
!docker push $image_name

We can now [test our image locally](04_Test_Locally.ipynb).